In [ ]:
import findspark
findspark.init('/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, col
from pyspark.sql.types import StructType, StructField, FloatType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
import time



spark = SparkSession.builder.appName("analytics").config("spark.driver.memory", "2g").getOrCreate()


# Streaming K-Means


### ¿En qué escenarios se puede aplicar?

* Deteccion de fraudes monitoreando transacciones
* Deteccion de anomalias en transito de red
* Monitoreo de sensores de IoT

## Generación de Datos sintéticos:

Para poder simular el flujo de información

In [ ]:




# Definimos el esquema del dataset simulado
schema = StructType([
    StructField("age", FloatType(), True),
    StructField("creatinine_phosphokinase", FloatType(), True),
    StructField("ejection_fraction", FloatType(), True),
    StructField("platelets", FloatType(), True),
    StructField("serum_creatinine", FloatType(), True),
    StructField("serum_sodium", FloatType(), True),
    StructField("time", FloatType(), True)
])

# Simulamos un stream de datos creando un DataFrame con valores aleatorios
def generate_stream_data():
    return spark.range(0, 100).select(
        (rand() * 100).cast("float").alias("age"),
        (rand() * 8000).cast("float").alias("creatinine_phosphokinase"),
        (rand() * 80).cast("float").alias("ejection_fraction"),
        (rand() * 450000).cast("float").alias("platelets"),
        (rand() * 5).cast("float").alias("serum_creatinine"),
        (rand() * 150).cast("float").alias("serum_sodium"),
        (rand() * 300).cast("float").alias("time")
    )

# Generamos datos de entrenamiento iniciales
initial_data = generate_stream_data()




## Procesamiento de datos

In [ ]:
# Seleccionamos las características que vamos a utilizar para el clustering
features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']

# Vectorizamos las características seleccionadas
vector_assembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")
initial_data_vector = vector_assembler.transform(initial_data)

# Escalamos las características
scaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
scaler_model = scaler.fit(initial_data_vector)
initial_data_scaled = scaler_model.transform(initial_data_vector)



## Entrenamiento del modelo


In [ ]:
# Entrenamos el modelo KMeans inicial
kmeans = KMeans(k=3, seed=1, featuresCol="features")
model = kmeans.fit(initial_data_scaled)



In [ ]:
# Función para transformar y escalar nuevos datos del stream
def transform_and_scale(df):
    df_vector = vector_assembler.transform(df)
    return scaler_model.transform(df_vector)

# Simulamos la generación de datos de stream y los escribimos a una carpeta
for i in range(2):
    stream_data = generate_stream_data()
    stream_data.write.mode("append").parquet("data/simulated_stream")
    time.sleep(1)



## Modelo de Streaming

In [ ]:
# Leemos el stream de datos desde la carpeta
streaming_data = spark.readStream.schema(schema).parquet("data/simulated_stream")

# Transformamos y escalamos los datos del stream
streaming_data_transformed = transform_and_scale(streaming_data)

# Realizamos el clustering en el stream de datos
streaming_predictions = model.transform(streaming_data_transformed)


## Streaming

Ejecucion del `StreamingQuery` para refrescar los datos como se generan.

Ejecutar el notebook "18-2" una vez que se ejecuta esta celda para que agregue datos al archivo que estamos utilizando para simular el streaming

In [ ]:

# Función para imprimir los centroides actuales
def print_centroids():
    centers = model.clusterCenters()
    print("Cluster Centers: ")
    for center in centers:
        print(center)

# Realizamos el clustering en el stream de datos y actualizamos el modelo
def update_and_print_clusters(df, epoch_id):
    global model
    df.persist()
    if not df.isEmpty():
        # Actualizamos el modelo con los nuevos datos del batch
        model = kmeans.fit(df)
        # Realizamos el clustering en el batch actual
        predictions = model.transform(df)
        # Imprimimos los centroides actuales
        centers = model.clusterCenters()
        print(f"Epoch {epoch_id} Cluster Centers: ")
        for center in centers:
            print(center)
        # Mostramos los resultados del clustering
#         predictions.show()
    df.unpersist()        
        
print("Centroides iniciales:")
print_centroids()
        
# Mostramos los resultados del clustering en tiempo real
# query = streaming_predictions.writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .trigger(processingTime='20 seconds') \
#     .foreachBatch(lambda df, epoch_id: print_centroids()) \
#     .start()
query = streaming_data_transformed.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='30 seconds') \
    .foreachBatch(update_and_print_clusters) \
    .start()

# Esperamos a que el stream termine
query.awaitTermination()